# Kernels
In this notebook we will compute the kernel (kernel as in kernel methods for non parametric regression, classification...) of a set of randomly sample set of 100 pixels.

Sample pixels an get them into a `numpy` array for further checks.

In [1]:
import ee
import numpy as np
from ee_ipl_uv import kernel

ee.Initialize()

# Get some bands of an image
im_original = ee.Image('LANDSAT/LC8_L1T_TOA/LC81980332015119LGN00')
properties = ["B4","B3","B2"]
im = im_original.select(properties)

# Sample 100 pixels of this 3 band image using seed 45. Resulting object is a featureCollection
feature_collection = im.sample(numPixels=100,seed=45)

# We have 100 features with 4 properties each (the properties are the bands of the image plus the index)
print("Number of features: ",feature_collection.size().getInfo())
print("Properties: ",ee.Feature(feature_collection.first()).propertyNames().getInfo())

# Construct kernel object
kernel_lineal = kernel.Kernel(feature_collection,properties)

# Convert local feature_collection to numpy array
array = kernel_lineal.getNumpy()

print("The local numpy retrieved array has shape: ",array.shape)


('Number of features: ', 100)
('Properties: ', [u'B4', u'B3', u'B2', u'system:index'])
('The local numpy retrieved array has shape: ', (100, 3))


In [2]:
# Select first element on the feature collection
current_array = ee.Feature(feature_collection.first()).toArray(properties)

print(current_array.getInfo())

[0.04522761330008507, 0.06556501984596252, 0.11201750487089157]


## Apply RBF kernel

Let's try an [rbf](https://en.wikipedia.org/wiki/Radial_basis_function_kernel) kernel. Which is also implemented our `kernel` module.

In [3]:

# Construct rbf kernel object with .5 gamma
kernel_rbf = kernel.Kernel(feature_collection,properties,distancia = kernel.RBFDistance(.5))

# Apply kernel (on GEE) to first feature
kernel_distances = kernel_rbf.applyToArray(current_array)

# Retrieve vector values as a numpy array
kernel_distances = np.asanyarray(kernel_distances.getInfo())

print("Retrieved values have shape:",kernel_distances.shape)

# Apply the rbf kernel with numpy:
first_value = array[0,:]
valor = np.zeros(array.shape[0])
for i in range(0,array.shape[0]):
    difference = array[i,:]-(first_value)
    valor[i] = np.exp(-.5*difference.dot(difference))

print("Applying the kernel with numpy lead to (almost) \
      the same results as doing it on the server: ",np.linalg.norm(valor-kernel_distances))

assert np.linalg.norm(valor-kernel_distances) < 1e-4, " computed kernel vector is not OK"

('Retrieved values have shape:', (100,))
('Applying the kernel with numpy lead to (almost)       the same results as doing it on the server: ', 8.1462152316096299e-09)


## Apply RBF kernel to set of Pixels

We will do it in python with `numpy` and `scipy` vectorized functions. 

In [4]:
from scipy.spatial.distance import pdist, squareform

# Compute the rbf kernel locally using scipy:
pairwise_dists = squareform(pdist(array, 'euclidean'))
kernel_matrix_rbf_numpy = np.exp(-.5*pairwise_dists**2)

# Check the local computation equals the loop implementation of above
print("The first column is as expected the array computed before: ",
      np.linalg.norm(valor-kernel_matrix_rbf_numpy[:,0]))

# Compute the kernel on GEE:
kernel_matrix_rbf = kernel_rbf.getKeeArray()
# Retrieve that kernel matrix as a numpy array
kernel_matrix_rbf = np.asanyarray(kernel_matrix_rbf.getInfo())

# Compare kernel computed locally and on GEE
print("We don't obtain the same kernel but approximately yes.",
      np.linalg.norm((kernel_matrix_rbf - kernel_matrix_rbf_numpy)))
assert np.linalg.norm((kernel_matrix_rbf - kernel_matrix_rbf_numpy)) < 1e-4," computed kernel matrix is not OK"

('The first column is as expected the array computed before: ', 1.5700924586837752e-16)
("We don't obtain the same kernel but approximately yes.", 1.1603736874027092e-07)


## Kernelized Ridge Regression

We want to try *krr* on GEE using our recently implemented *rbf* kernel. 

To do this we will take another 100 values from `B5` band: $y \in \mathbb{R}(p)$ of the same pixels (we will achieve this using the same seed on sample). Then we will fit the model:
$$
 y_i \sim \phi(x_i)^t\cdot w
$$

### Model Description

Where $\phi$ is the non-linear transform induced by rbf kernel. Therefore we want to minimize:

$$
|| y - \phi(X)\cdot w ||^2 + \lambda|| w ||^2
$$

Where we have introduced the regularization term: $\lambda|| w ||^2$. This leads to the optimum $w^\star$:
$$
w^\star = \phi(X)^t (\phi(X)\cdot \phi(X)^t+\lambda I )^{-1}  \cdot y
$$

Using that our kernel matrix $K=\phi(X)\cdot \phi(X)^t$ and naming $\alpha = (K+\lambda I )^{-1}  \cdot y \in \mathbb{R}(p)$ we have:

$$
w^\star = \phi(X)^t\cdot \alpha
$$

Finally, we can apply our model to a new sample $x'\in \mathbb{R}(B)$ which will be:

$$
\phi(x')^t \cdot w^\star = k'^t \cdot \alpha
$$

Where $k' = \phi(x')^t \cdot \phi(X)^t$ corresponds to apply the kernel to the feature vector $x'$.

We start by selecting the band, checking that same seed means same sampled `featureCollection`.

In [5]:
# random_vector_server = ee.Array(np.random.randn(array.shape[0]).tolist())

# Select 4 and 5 layer
vector_server = im_original.select("B[4-5]")

# Sample 100 pixels from this layer
vector_server = vector_server.sample(numPixels=100,seed=45)

# Check B4 pixels are equal to the ones on our array (should be as we are using the same seed)
B4_vector = np.asanyarray(vector_server.aggregate_array("B4").getInfo())
print("B4 pixels are the same as before: ",np.all(B4_vector == array[:,0]))

# Get B5 vector
B5_vector = ee.Array(vector_server.aggregate_array("B5"))

# We do not retrieve B5:
print(type(B5_vector))

# IMPORTANT: B5 must be 2 dim array!!
B5_vector = ee.Array.cat([B5_vector], 1)

print(B5_vector.length().getInfo())


('B4 pixels are the same as before: ', True)
<class 'ee.Array'>
[100, 1]


We will retrieve the $\alpha$ vector using a regularization factor $\lambda = .1$

In [6]:
lda = .1
np_alpha_vector = kernel_rbf.getAlphaNumpy(B5_vector,lda)

np_alpha_vector.shape

(100, 1)

Check alpha vector using `numpy`.

In [7]:
# Convert B5 server vector to numpy array
B5_vector_numpy = np.asanyarray(B5_vector.getInfo())

# Compute alpha using numpy (K+\lamdaI)^-1*y
alpha_local = np.linalg.solve(kernel_matrix_rbf_numpy+lda*np.eye(kernel_matrix_rbf_numpy.shape[0]),B5_vector_numpy)
print("Shape of alpha vector is:",alpha_local.shape)
print(" alpha local vs alpha server diff:",np.linalg.norm((alpha_local - np_alpha_vector)))
assert np.linalg.norm((alpha_local - np_alpha_vector)) < 1e-4, " alpha local vs alpha server diff are OK"
print("Norm of alpha (big values usually means problems):",np.linalg.norm(alpha_local))

('Shape of alpha vector is:', (100, 1))
(' alpha local vs alpha server diff:', 5.6272754855707717e-07)
('Norm of alpha (big values usually means problems):', 5.3050682226632162)


### Compute Residuals
We know compare the *RMSE* of the model on the in sample values (in-sample error). The computed *RMSE* is lower than the standard deviation (sd) which is the error using the constant mean as the model. 

In [8]:
residuals = kernel_matrix_rbf_numpy.dot(alpha_local)-B5_vector_numpy
print("Shape of residuals vector:",residuals.shape)
print("RMSE:",np.sqrt(np.mean(residuals**2)))

print("Percentiles: ",np.percentile(B5_vector_numpy,[0,5,25,50,75,95,100]))
print("sd: ",np.std(B5_vector_numpy))


('Shape of residuals vector:', (100, 1))
('RMSE:', 0.053050682226632354)
('Percentiles: ', array([ 0.01414374,  0.01669631,  0.03180608,  0.08420379,  0.23319263,
        0.46967739,  0.82202411]))
('sd: ', 0.16169343573704986)


### Apply kernelized regression to a Image

This task will be attempted in `kernels_part_2.ipynb` notebook.